In [ ]:
import tensorflow as tf 
print(tf.__version__)

In [ ]:
import keras_nlp 
import tensorflow as tf 
import tensorflow_datasets as tfds 
import sklearn.cluster as cluster 
from tensorflow import keras  
# Thiết lập chính sách độ chính xác toàn cục cho các lớp keras 
# 1 Xác định kiểu tính toán và kiểu dữ liệu biến cho một lớp hoặc
# một mô hình sử dựng float 16 cho các phép tính và float 32 cho các biến 
# nhằm tăng tốc độ huấn luyện và giảm bộ nhớ trên GPU
policy = keras.mixed_precision.Policy("mixed_float16")
keras.mixed_precision.set_global_policy(policy)

Dowload Dataset

In [ ]:
TRAIN_BATCH_SIZE = 6
VALIDATION_BATCH_SIZE = 8 

TRAIN_NUM_BATCHS = 300 
VALIDATION_NUM_BATCHS = 40

AUTOTUNE = tf.data.experimental.AUTOTUNE

# Xây dựng phương thức chage_range  
# sử dụng để chuyền nhãn của tập dữ liệu [0 ,5] -> [-1 , 1]
# nhằm phù hợp với hàm mất mát của mô hình 
def change_range(x):
    return (x / 2.5) - 1

# Xây dựng phương thức Prepare_dataset để chuẩn bị cho 
# việc huấn luyện mạng siamese , nhận đầu vào tập dữ liệu thô , 
# số lượng lô , và kích thước lô 
def prepare_dataset(dataset , num_batchs , batch_size):
    # áp dụn hàm map để biến đổi mỗi phần tử trong tập dữ liệu thành một cặp    
    # gồm danh sách 2 câu và danh sách một bộ nhãn được chuyển đổi 
    dataset = dataset.map(
        lambda z : (
            [z['sentence1'], z['sentence2']],
            [tf.cast(change_range(z['label']), tf.float32)]
        ), 
        num_parallel_calls=AUTOTUNE,
    )
    # áp dụng batch để lấy ra kích thước batch 
    dataset = dataset.batch(batch_size)
    # áp dụng hàm take để lấy số lượng batch theo batch_size 
    dataset = dataset.take(num_batchs)
    # áp dụng hàm prefectch để tải các các tiếp theo trong khi xử lý các 
    # batch hiện tại áp dụng tham số autotune để tự động điều chỉnh 
    # số lượng batch được tiền tải 
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset 

# Tải tập dữ liệu glue/stsb
stsb_ds = tfds.load(
    "glue/stsb",
)
# Lấy ra dữ liệu train và vlidation từ data glue/stsb đã load trước đó 
stsb_train , stsb_valid = stsb_ds['train'],  stsb_ds['validation']

# Biến đổi 2 bộ dữ liệu train và validation thô thành tiêu chuẩn để có thể 
# sử dụng cho mô hình , và truyền vào đó các tham số cần thiết 
stsb_train  = prepare_dataset(stsb_train , TRAIN_NUM_BATCHS , TRAIN_BATCH_SIZE)

stsb_valid = prepare_dataset(stsb_valid, VALIDATION_NUM_BATCHS, VALIDATION_BATCH_SIZE)


# In ra các mẫu từ data train 
for x , y in stsb_train:
    for i , example in enumerate(x):
        print(f"sentence 1 : {example[0]} ")
        print(f"sentence 2 : {example[1]} ")
        print(f"similarity : {y[i]} \n")
    break

Build the encoder model 

In [ ]:
# Một lớp tiền sử lý để mã hóa và tạo mặt nạ đệm cho các câu 
preprocessor = keras_nlp.models.RobertaPreprocessor.from_preset("roberta_base_en")
# Một lớp kiến trúc xương sống tạo ra sự hiểu biết biểu diễn theo ngữ cảnh 
backbone = keras_nlp.models.RobertaBackbone.from_preset("robert_base_en")
# Thêm 1 lớp đầu vào dạng chuỗi shape = 1 là cho phép 1 câu văn bản 
inputs = keras.Input(shape=(1), dtype="string" , name='sentence')
# Thêm 1 lớp tiền sử lý đầu vào 
x = preprocessor(inputs)
# đưa dữ liệu đã xử lý qau mô hình kiến trúc backbone
h = backbone(x) 
# Thêm 1 lớp gộp Pooling nhận đầu vào h và x với padding mask x 
embedding = keras.layers.GlobalAveragePooling1D(name="pooling_layer")(
    h, x["padding_mask"]
)
# áp dụng chuẩn hóa theo trục để chuẩn hóa vector đầu ra theo chuẩn EUCLIDEAN 
# Trả về 2 giá trị vector chuẩn hóa và giá trị chuẩn hóa của vector ban đầu 
n_embedding = tf.linalg.normalize(embedding , axis=1)[0] # lấy vector đã được chuẩn hóa
roberta_normal_encoder = keras.Model(inputs=inputs , outputs = n_embedding)
roberta_normal_encoder.summary()

Build the Siamese network with the regression objective funcion

In [ ]:
# Xây dựng mạng siamese hồi quy sử dụng mô hình mã hóa Robert
# để tính toán độ tương đồng cosin giữa 2 văn bản '
class RegressionSiamese(keras.Model):
    # xây dựng phương thức khởi tạo nhận vào là khối mã hóa của Robert 
    # và **kwargs là tham số bất kỳ 
    def __init__(self, encoder , **kwargs):
        # Thiết lập đầu vào dạng chuỗi với 2 câu 
        inputs = keras.Input(shape=(2), dtype='string', name='sentences')
        # tách ra thành 2 câu riêng lẻ để có thể tính toán độ tương tự 
        # cosin , tách theo trục 
        sen1 , sen2 = tf.split(inputs , num_or_size_splits=2, axis=1 , name='split')
        # Thực hiện mã hóa sen1  và 2 bởi Robert encoder 
        u = encoder(sen1)
        v = encoder(sen2)
        # Tính toán tích vô hướng (tích gộp) giữa u và v chuyển vị
        # kết quả là 1 tensor shape = [batch_size , batch_size]
        # chưa các giá trị độ tương đồng cosin giữa các cặp câu văn bản trong batch 
        cosine_similarity_scores = tf.matmul(u, tf.transpose(v))

        # gọi lại phương thức khởi tạo của lớp cha 
        # và chyền vào các tham số 
        super().__init__(
            inputs=inputs,
            outputs=cosine_similarity_scores,
            **kwargs,
        )
        # khởi tạo encoder 
        self.encoder = encoder 

    # Xây dựng phuuwong thức trả về bộ phận mã hóa 
    def get_encoder (self):
        return self.encoder 

Fit the model 

In [ ]:
# tạo một bộ từ điển với danh sách chưa  câu 
sentences = [
    "Today is a very sunny day.",
    "I am hungry, I will get my meal.",
    "The dog is eating his food.",
]
# tạo một danh sách chưa câu truy vấn 
# để so sánh độ tươnh đồng về ngữ nghĩa với các cau trong từ điển trên
query = ['The dog is enjoying his meal.']
# đặt encoder = Robert model 
encoder = roberta_normal_encoder 

# thực hiện mã hóa bộp từ điển và câu truy vấn sử dụng tf.constant để 
# không thay đổi giá trị 
sentence_embeddings = encoder(tf.constant(sentences))
query_embedding = encoder(tf.constant(query))

# Tính toán độ tương tự cosin sử dụng hàm tf.matmul để tính tích vô hướng 
cosine_similarity_scores = tf.matmul(query_embedding, tf.transpose(sentence_embeddings))

# Duyệt qua các cặp một và in ra danh sách cùng với độ tương đồng của nó
for i , sim in enumerate(cosine_similarity_scores[0]):
    print(f"cosine similarity score between sentence {i+1} and the query = {sim} ")

Train the model 

In [ ]:
# Khởi tạo mô hình mạng siamese hồi quy robert 
roberta_regression_siamese = RegressionSiamese(roberta_normal_encoder)

# Trình biên dịch mô hình 
roberta_regression_siamese.compile(
    # ,loss = MeanSquaredError 
    loss=keras.losses.MeanSquaredError(),
    # trình tối ưu hóa = Adam lr = 2e-5 
    optimizer=keras.optimizers.Adam(2e-5),
)
roberta_regression_siamese.fit(stsb_train, validation_data=stsb_valid, epochs=1)

Test affter Training model 

In [ ]:
sentences = [
    "Today is a very sunny day.",
    "I am hungry, I will get my meal.",
    "The dog is eating his food.",
]
query = ["The dog is enjoying his food."]

encoder = roberta_regression_siamese.get_encoder()

sentence_embeddings = encoder(tf.constant(sentences))
query_embedding = encoder(tf.constant(query))

cosine_simalarities = tf.matmul(query_embedding, tf.transpose(sentence_embeddings))
for i, sim in enumerate(cosine_simalarities[0]):
    print(f"cosine similarity between sentence {i+1} and the query = {sim} ")

Fine-tune Using the triplet Objective Function
*  Load the data

In [ ]:
!wget https://sbert.net/datasets/wikipedia-sections-triplets.zip -q
!unzip wikipedia-sections-triplets.zip  -d  wikipedia-sections-triplets

In [ ]:
NUM_TRAIN_BATCHS = 200
NUM_TEST_BATCHS = 75 
AUTOTUNE = tf.data.experimental.AUTOTUNE 

# Xây dựng phương thức tách dữ liệu và dựng giữ liệu từ wiki 
def prepare_wiki_data(dataset , num_batchs):
    # tách ra thành 3 câu từ bộ dữ liệu 
    datasetc= dataset.map(
        # 0 được sử dụng như một nhãn giả cho việc huấn luyện và kiểm tra 
        lambda z: ((z['Sentence1'], z['Sentence2'] , z['Sentence3']),0)
    )
    # Đặt số lô , kích thước cho mỗi batch, và phép biến đổi prefetch 
    dataset = dataset.batch(6)
    dataset = dataset.take(num_batchs)
    # áp dụng hàm prefectch để tải các các tiếp theo trong khi xử lý các 
    # batch hiện tại áp dụng tham số autotune để tự động điều chỉnh 
    # số lượng batch được tiền tải 
    dataset = dataset.prefecth(AUTOTUNE)


# Lấy ra bộ wiki train và test 
wiki_train = tf.data.experimental.make_csv_dataset(
     "wikipedia-sections-triplets/train.csv",
    batch_size=1,
    num_epochs=1,
)
wiki_test = tf.data.experimental.make_csv_dataset(
    "wikipedia-sections-triplets/test.csv",
    batch_size=1,
    num_epochs=1,
)

# Tạo bộ dữ liệu train , test hoàn  chỉnh 
wiki_train = prepare_wiki_data(wiki_train , NUM_TRAIN_BATCHS)
wiki_test = prepare_wiki_data(wiki_test, NUM_TEST_BATCHS)

Build the Encoder Model 

In [ ]:
# Thiết lập lớp xử lý với Robert
preprocessor = keras_nlp.models.RobertaPreprocessor.from_preset("roberta_base_en")
# Thêm lớp Backbone Robert 
backbone = keras_nlp.models.RobertBackbone.from_preset("roberta_base_en")
# Thiết lập lơp xử lý đầu vào 
input = keras.Input(shape=(1), dtype='string', name="sentence")

x = preprocessor(input)
h = backbone(x)
embedding = keras.layers.GlobalAveragePooling1D(name="pooling_layer")(
    h, x["padding_mask"]
)

roberta_encoder = keras.Model(inputs=input, outputs=embedding)
roberta_encoder.summary()


Build the Siamese network with the triplet objective funcion 

In [ ]:
# xây dựng mạng siamese với hàm mục tiêu triplet loss 
# tính khoảng cách của negative và positive 
class TripletSiamese(keras.Model):
    def __init__(self, encoder , **kwargs):

        anchor = keras.Input(shape=(1) , dtype='string')
        positive = keras.Input(shape=(1), dtype='string')
        negative = keras.Input(shape=(1) , dtype='string')

        ea = encoder(anchor)
        ep = encoder(positive)
        en = encoder(negative)

        positive_dist = tf.math.reduce_sum(tf.math.pow(ea - ep, 2), axis=1)
        negative_dist = tf.math.reduce_sum(tf.math.pow(ea - en , 2), axis=1)

        positive_dist = tf.math.sqrt(positive_dist)
        negative_dist = tf.math.sqrt(negative_dist)

        outputs = tf.stack([positive_dist , negative_dist] , axis=0)

        super().__init__(
            inputs = [anchor , positive , negative],
            outputs = outputs , 
            **kwargs 
        )
        self.encoder = encoder 

    def get_encoder(self):
        return self.encoder

Buil Triplet loss

In [ ]:
class TripletLoss(keras.losses.Loss):
    def __init__(self, margin=1 , **kwargs):
        super().__init__(**kwargs)
        self.margin = margin 

    def call(self, y_true , y_pred):
        positive_dist , negative_dist = tf.unstack(y_pred , axis=0)

        losses = tf.nn.relu(positive_dist - negative_dist + self.margin)
        return tf.math.reduce_mean(losses , axis=0)
    

Fit the model

In [ ]:
roberta_triplet_siamese = TripletSiamese(roberta_encoder)

roberta_triplet_siamese.compile(
    loss=TripletLoss(),
    optimizer=keras.optimizers.Adam(2e-5),
)

roberta_triplet_siamese.fit(wiki_train, validation_data=wiki_test, epochs=1)

In [ ]:
# Tạo một tập từ điển chứa 1 danh sách gồm 6 câu hỏi . 
questions = [
    "What should I do to improve my English writting?",
    "How to be good at speaking English?",
    "How can I improve my English?",
    "How to earn money online?",
    "How do I earn money online?",
    "How to work and ean money through internet?",
]
# gọi ra lớp mã hóa từ Robert 
encoder = roberta_triplet_siamese.get_encoder()
embeddings = encoder(tf.constant(questions))
# Phân loại với Kmeans 
kmeans = cluster.KMeans(n_clusters=2, random_state=0, n_init="auto").fit(embeddings)

for i, label in enumerate(kmeans.labels_):
    print(f"sentence ({questions[i]}) belongs to cluster {label}")